In [1]:
import numpy as np

import sys
sys.path.append("/remote/gpu03/schiller/skatr")
# sys.path.append("..")
# sys.path.append("/home/hd/hd_hd/hd_ps236/skatr/")

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

In [9]:
from src.networks.llm_vit_construct.modeling_llm_vit import LLMViTForCausalLM
from src.networks.llm_vit_construct.configuration_llm_vit import LLMViTConfig

if 'model' in locals():
    del model
model = LLMViTForCausalLM(LLMViTConfig(device_map="cpu"))

In [10]:
import os, json
from transformers.utils.__init__ import SAFE_WEIGHTS_INDEX_NAME
from transformers.modeling_utils import load_state_dict, set_module_tensor_to_device
from accelerate.utils.modeling import find_tied_parameters
# from safetensors.torch import load_file
import gc
import fnmatch
import torch

# data_dir = "/home/hd/hd_hd/hd_ps236/skatr/models/huggingface_models/Phi3_5_v"
data_dir = "/remote/gpu03/schiller/skatr/models/huggingface_models/Phi3_5_v"

archive_file = os.path.join(data_dir, SAFE_WEIGHTS_INDEX_NAME)
modules_to_load = [
    "lm_head.weight",
    "model.layers.*",
    "model.norm.weight"
]

def _to_be_loaded(name:str, modules_to_load:list[str]) -> False:
    for module in modules_to_load:
        if fnmatch.fnmatch(name, module):
            return True
    return False

with open(archive_file, "r") as f:
    index = json.loads(f.read())

shard_filenames = sorted(set(index["weight_map"].values()))
sharded_metadata = index["metadata"]
sharded_metadata["all_checkpoint_keys"] = list(index["weight_map"].keys())
sharded_metadata["weight_map"] = index["weight_map"].copy()
shard_filenames = [os.path.join(data_dir, f) for f in shard_filenames]
loaded_state_dict_keys = sharded_metadata["all_checkpoint_keys"]

model.tie_weights()
model_state_dict = model.state_dict()
_expected_keys = list(model_state_dict.keys())
expected_keys = []
for key in _expected_keys:
    for module in modules_to_load:
        if fnmatch.fnmatch(key, module):
            expected_keys.append(key)
            break

loaded_keys = [key for key in loaded_state_dict_keys]
prefix = model.base_model_prefix

missing_keys = sorted(set(expected_keys) - set(loaded_keys))
unexpected_keys = set(loaded_keys) - set(expected_keys)

tied_params = find_tied_parameters(model)

for group in tied_params:
    missing_in_group = [k for k in missing_keys if k in group]
    if len(missing_in_group) > 0 and len(missing_in_group) < len(group):
        missing_keys = [k for k in missing_keys if k not in missing_in_group]

not_initialized_submodules = dict(model.named_modules())
dtype=torch.float32

for shard_file in shard_filenames:
    state_dict = load_state_dict(shard_file, False)

    state_dict_to_load = {k: v for k,v in state_dict.items() if _to_be_loaded(k, modules_to_load)}
    model.load_state_dict(state_dict_to_load, strict=False)

    # for param_name, param in state_dict.items():

    #     if param_name not in expected_keys:
    #         continue

    #     module_name = param_name
    #     set_module_kwargs = {}
        
    #     param = param.to(dtype)

    #     old_param = model
    #     splits = param_name.split(".")
    #     for split in splits:
    #         old_param = getattr(old_param, split)
    #         if old_param is None:
    #             break
    #     if old_param is not None:
    #         if dtype is None:
    #             param = param.to(old_param.dtype)

    #         if old_param.is_contiguous():
    #             param = param.contiguous()

    #     set_module_kwargs["value"] = param
    #     set_module_kwargs["dtype"] = torch.float32
    #     set_module_tensor_to_device(model, param_name, "cpu", **set_module_kwargs)

    del state_dict
    gc.collect()

In [1]:
from pdf2image import convert_from_path
pages = convert_from_path('recovery.pdf')
pages[0].save('image.jpg', 'JPEG')

In [ ]:
import sys
sys.path.append(str("/remote/gpu03/schiller/skatr/models/huggingface_models/Phi3_5_v"))
from skatr_vit_phi3_v import PretrainedViT

vit_model = PretrainedViT(backbone_dir="/remote/gpu03/schiller/skatr/runs/regression_micro_ViT/2024-10-02_12-00-27")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoTokenizer, pipeline, BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(load_in_4bit=True)

if 'model' in locals():
    del model
model = AutoModelForCausalLM.from_pretrained( 
    # "/remote/gpu03/schiller/ExecLLM/models/huggingface/microsoft/Phi-3.5-MoE-instruct",
    # "/remote/gpu03/schiller/skatr/models/huggingface_models/Phi3_5_v",
    "/home/hd/hd_hd/hd_ps236/skatr/models/huggingface_models/Phi3_5_v",
    # device_map="cuda",
    device_map="cpu", 
    torch_dtype=torch.bfloat16,  
    trust_remote_code=True,
    # quantization_config=quantization_config,
    # attn_implementation="flash_attention_2",
    _attn_implementation="eager",
    ignore_mismatched_sizes=True,
    output_loading_info=True,
)
    
# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-MoE-instruct", local_files_only=True) 
# tokenizer = AutoTokenizer.from_pretrained("/remote/gpu03/schiller/ExecLLM/models/huggingface/microsoft/Phi-3.5-vision-instruct", local_files_only=True) 

/home/hd/hd_hd/hd_ps236/.conda/envs/skatr/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [10:53<00:00, 326.92s/it]


In [ ]:
import sys
sys.path.append("/remote/gpu03/schiller/skatr/models/huggingface_models/Phi3_5_v")
from skatr_vit_phi3_v import PretrainedViT, Phi3ImageEmbeddingSkatr

model[0].config.img_processor = {
    "image_dim_out": 144,
    "model_name": "",
    "name": "vit",
    "num_img_tokens": 144,
    "backbone_dir": "/remote/gpu03/schiller/skatr/runs/regression_micro_ViT/2024-10-02_12-00-27"
    }
embedding_config = {
    'embedding_cls': model[0].config.embd_layer['embedding_cls'],
    **model[0].config.embd_layer
    }
model[0].model.vision_embed_tokens = Phi3ImageEmbeddingSkatr(
    model[0].config,
    wte=model[0].model.embed_tokens,
    **embedding_config
    )
# model[0].model.vision_embed_tokens.img_processor = PretrainedViT(backbone_dir="/remote/gpu03/schiller/skatr/runs/regression_micro_ViT/2024-10-02_12-00-27")
# model[0].config.embd_layer['use_hd_transform'] = False
# model[0].config.embd_layer['with_learnable_separator'] = False

In [5]:
model[0].save_pretrained(save_directory="/remote/gpu03/schiller/skatr/models/custom/Phi3_v_skatr", safe_serialization=False)

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(
    "/remote/gpu03/schiller/skatr/models/huggingface_models/Phi3_5_v", 
    trust_remote_code=True,
    local_files_only=True,
    patch_size = [4, 4, 10]
)
processor.save_pretrained("/remote/gpu03/schiller/skatr/models/test")

In [ ]:
import sys
sys.path.append(str("/remote/gpu03/schiller/skatr/models"))

from convert_phi3_to_skatr import convert_phi3_to_skatr

import shutil

import torch
from transformers import AutoModelForCausalLM

if 'model' in locals():
    del model

load_dir = "/remote/gpu03/schiller/skatr/models/huggingface_models/Phi3_5_v"
model = AutoModelForCausalLM.from_pretrained( 
    # "/remote/gpu03/schiller/ExecLLM/models/huggingface/microsoft/Phi-3.5-MoE-instruct",
    "/remote/gpu03/schiller/skatr/models/huggingface_models/Phi3_5_v",
    device_map="cuda",  
    torch_dtype=torch.bfloat16,  
    trust_remote_code=True,
    # quantization_config=quantization_config,
    # attn_implementation="flash_attention_2",
    ignore_mismatched_sizes=False,
    output_loading_info=True,
)

model, processor = convert_phi3_to_skatr(model[0], patch_size=[4,4,10])

print(f"{processor.get_lightcone_param_token()} = {processor.lightcone_param_token_id()}")

save_dir="/remote/gpu03/schiller/skatr/models/custom/Phi3_v_skatr"
shutil.rmtree(save_dir)

model.save_pretrained(save_directory=save_dir, safe_serialization=False)
processor.save_pretrained(save_directory=save_dir)

files_to_copy = ["modeling_phi3_v.py", "skatr_vit_phi3_v.py"]
for file in files_to_copy:
    shutil.copy(f"{load_dir}/{file}", f"{save_dir}/{file}")

In [ ]:
from src.networks.llm_vit_construct.modeling_llm_vit import LLMViTForCausalLM
from src.networks.llm_vit_construct.configuration_llm_vit import LLMViTConfig

if 'model' in locals():
    del model

model = LLMViTForCausalLM(LLMViTConfig(trust_remote_code=True, local_files_only=True,))


In [ ]:
from transformers import AutoModelForCausalLM, AutoProcessor
import shutil, os

if 'model' in locals():
    del model

# load_dir = "/remote/gpu03/schiller/skatr/src/networks/llm_vit_construct"
# load_dir = "../src/networks/llm_vit_construct"
load_dir = "/home/hd/hd_hd/hd_ps236/skatr/src/networks/llm_vit_construct"

model = AutoModelForCausalLM.from_pretrained(load_dir, trust_remote_code=True, local_files_only=True)
model.load_vit_weights("vit_default.pt")
processor = AutoProcessor.from_pretrained(load_dir, trust_remote_code=True, local_files_only=True)

# save_dir="/remote/gpu03/schiller/skatr/src/networks/llm_vit"
# save_dir="../src/networks/llm_vit"
save_dir = "/home/hd/hd_hd/hd_ps236/skatr/src/networks/llm_vit"
if os.path.isdir(save_dir):
    shutil.rmtree(save_dir)

model.save_pretrained(save_directory=save_dir)
processor.save_pretrained(save_directory=save_dir)


In [ ]:
import numpy as np

data_dir = '/remote/gpu02/ore/data/x5'
index = 1000
data_file = data_dir + f"/run{index}.npz"
data = np.load(data_file)
lightcone = data['image']

images = [np.asarray(lightcone)]
placeholder = f"<|lightcone_1|>"

messages = [
    {"role": "user", "content": placeholder+processor.get_lightcone_param_token()},
]

prompt = processor.tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=False
)

inputs = processor(prompt, images, return_tensors="pt").to("cuda:0")
output = model(**inputs, output_hidden_states=True, return_dict=True)

In [ ]:
param_mask = (inputs.input_ids[0,:] == processor.lightcone_param_token_id())
param_positions = param_mask.nonzero(as_tuple=True)[0]
param_pos = param_positions[0]

print(output.hidden_states[:,param_pos])

In [ ]:
from PIL import Image 

images = [Image.open("image.jpg")]
placeholder = f"<|image_1|>"

messages = [
    {"role": "user", "content": placeholder+"Summarize this image."},
]

prompt = processor.tokenizer.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)

inputs = processor(prompt, images, return_tensors="pt").to("cuda:0") 

generation_args = { 
    "max_new_tokens": 1000, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

generate_ids = model.generate(
    **inputs, 
    eos_token_id=processor.tokenizer.eos_token_id, 
    **generation_args
)

# remove input tokens 
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(
    generate_ids, 
    skip_special_tokens=True, 
    clean_up_tokenization_spaces=False)[0] 

print(response)

In [ ]:
import numpy as np

data_dir = '/remote/gpu02/ore/data/x5'
index = 1000
data_file = data_dir + f"/run{index}.npz"
data = np.load(data_file)
print(data['image'].shape)

In [ ]:
import json

data_dict = {'image': data['image'].tolist(), 'label': data['label'].tolist()}
# data_dict = {'image': data['image'][:10,:10,550:600].tolist(), 'label': data['label'].tolist()}
data_str = json.dumps(data_dict)
# print(data_str[0:1000])

In [ ]:
messages = [
    {"role": "user", "content": data_str},
]

prompt = processor.tokenizer.apply_chat_template(
  messages, 
  tokenize=True,
  add_generation_prompt=True
)
print(len(prompt))